In [205]:
import gensim
import nltk
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Embedding, TimeDistributed, RepeatVector, Input
from keras.callbacks import ModelCheckpoint
from keras.utils.np_utils import to_categorical
import pandas as pd
import numpy as np

### Parameters

In [194]:
filename = 'test.txt'

IN_SEQ_LENGTH = 3
OUT_SEQ_LENGTH = 3
EMBEDDING_DIM = 4
HIDDEN_LAYER_1 = 10

N_EPOCH = 10

path_train_X = '../data/' + filename + '.' + str(IN_SEQ_LENGTH) + '_in_' + str(OUT_SEQ_LENGTH) + '_out.train_X.csv'
path_train_Y = '../data/' + filename + '.' + str(IN_SEQ_LENGTH) + '_in_' + str(OUT_SEQ_LENGTH) + '_out.train_Y.csv'

### Loading corpora

In [208]:
corpora = gensim.corpora.Dictionary.load('../data/'+filename+'.corpora.dat')
vocab_size = len(corpora)
print('Number of words in corpora: %d'%(vocab_size))
tmp = list(corpora.items())

META_UNKNOWN = '<<<!UNK!>>>'
META_EMPTY = '<<<!EMP!>>>'
META_NUMBER = '<<<!NUM!>>>'

Number of words in corpora: 29


### Loading training dataset

In [195]:
train_X = pd.read_csv(path_train_X, dtype=np.int32)
train_Y = pd.read_csv(path_train_Y, dtype=np.int32)

In [196]:
print('Train set X, Y:')
print( train_X.shape, train_Y.shape )

Train set X, Y:
(37, 3) (37, 3)


In [197]:
# convert Y to sparce matrix

In [198]:
out_Y = np.zeros((train_Y.shape[0], OUT_SEQ_LENGTH, vocab_size), dtype=np.int32)

In [199]:
out_Y = np.expand_dims(train_Y.as_matrix(), -1)

### Set network configureation

In [200]:
model = Sequential()
model.add( Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, input_length=IN_SEQ_LENGTH) )
model.add( LSTM(HIDDEN_LAYER_1, return_sequences=True) )
model.add( TimeDistributed(Dense(output_dim=vocab_size, activation='sigmoid')) )

In [201]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [202]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_29 (Embedding)         (None, 3, 4)          116         embedding_input_27[0][0]         
____________________________________________________________________________________________________
lstm_28 (LSTM)                   (None, 3, 10)         600         embedding_29[0][0]               
____________________________________________________________________________________________________
timedistributed_15 (TimeDistribu (None, 3, 29)         319         lstm_28[0][0]                    
Total params: 1,035
Trainable params: 1,035
Non-trainable params: 0
____________________________________________________________________________________________________


In [203]:
print( model.layers[0].output_shape )
print( model.layers[1].output_shape )
print( model.layers[2].output_shape )

(None, 3, 4)
(None, 3, 10)
(None, 3, 29)


In [204]:
#model.fit(train_X.as_matrix(), train_Y.as_matrix(), batch_size=500, nb_epoch=N_EPOCH)
model.fit(train_X.as_matrix(), out_Y, batch_size=500, nb_epoch=N_EPOCH)

Epoch 1/10
37/37 [==============================] - 0s - loss: 3.3673 - acc: 0.0631
Epoch 2/10
37/37 [==============================] - 0s - loss: 3.3669 - acc: 0.0450
Epoch 3/10
37/37 [==============================] - 0s - loss: 3.3665 - acc: 0.0541
Epoch 4/10
37/37 [==============================] - 0s - loss: 3.3661 - acc: 0.0541
Epoch 5/10
37/37 [==============================] - 0s - loss: 3.3657 - acc: 0.0811
Epoch 6/10
37/37 [==============================] - 0s - loss: 3.3653 - acc: 0.0721
Epoch 7/10
37/37 [==============================] - 0s - loss: 3.3649 - acc: 0.0811
Epoch 8/10
37/37 [==============================] - 0s - loss: 3.3645 - acc: 0.0811
Epoch 9/10
37/37 [==============================] - 0s - loss: 3.3641 - acc: 0.0811
Epoch 10/10
37/37 [==============================] - 0s - loss: 3.3637 - acc: 0.0811


In [146]:
#xin = Input(batch_shape=(train_X.shape[0], IN_SEQ_LENGTH), dtype='int32')
#xemb = Embedding(vocab_size, EMBEDDING_DIM)(xin) # 3dim (batch,time,feat)
#seq = LSTM(IN_SEQ_LENGTH, return_sequences=True)(xemb)
#mlp = TimeDistributed(Dense(OUT_SEQ_LENGTH, activation='softmax'))(seq)
#model = Model(input=xin, output=mlp)
#model.compile(optimizer='Adam', loss='categorical_crossentropy')

In [206]:
def get_last_token_ids(inp, seq_len):
    
    res = np.full(seq_len, corpora.token2id[META_EMPTY], dtype=int)
      
    raw_sents = nltk.tokenize.sent_tokenize(inp.lower())

    id_tokens = []
    
    for s in raw_sents:
        raw_tokens = nltk.tokenize.wordpunct_tokenize(s)
       
        for t in raw_tokens:
            try:
                tid = corpora.token2id[t]
            except:
                tid = corpora.token2id[META_UNKNOWN]
            id_tokens.append(tid)
            #print('%s -> %d'%(t, tid))

    l = min(seq_len, len(id_tokens))
    s = seq_len - l
    #print(id_tokens)
    #print(id_tokens[-l:])
    
    res[s:] = id_tokens[-l:]
       
    return res

In [209]:
get_last_token_ids('One two', IN_SEQ_LENGTH)

array([27,  3,  5])

In [332]:
def get_next_sequence(inp):
    tids = get_last_token_ids(inp, IN_SEQ_LENGTH)
    
    print(tids.shape)
   
    arrs = np.array(tids)[np.newaxis,:]
    
    print(arrs.shape)
    
    p = model.predict(arrs)
    
    print('p.shape: {}'.format(p.shape))
    
    print('p: {}'.format(p))
    
    w1ind = np.argmax(p[0,0,:])
    w2ind = np.argmax(p[0,1,:])
    w3ind = np.argmax(p[0,2,:])
    
    print(corpora.id2token[w1ind])
    print(corpora.id2token[w2ind])
    print(corpora.id2token[w3ind])
    
    #return m0 #p0 = p[m0]; p[m0] = 0
    #m1 = np.argmax(p); p1 = p[m1]; p[m1] = 0
    #m2 = np.argmax(p); p2 = p[m2]; p[m2] = 0
    #m3 = np.argmax(p); p3 = p[m3]; p[m3] = 0
    #m4 = np.argmax(p); p4 = p[m4]; p[m4] = 0
    #m5 = np.argmax(p); p5 = p[m5]; p[m5] = 0
    #m6 = np.argmax(p); p6 = p[m6]; p[m6] = 0
    
    #return [(p0, corpora.id2token[m0]),
    #        (p1, corpora.id2token[m1]),
    #        (p2, corpora.id2token[m2]),
#            (p3, corpora.id2token[m3]),
#           p4, corpora.id2token[m4]),
#          (p5, corpora.id2token[m5]),
#         (p6, corpora.id2token[m6])]

In [334]:
get_next_sequence('One two three')

(3,)
(1, 3)
p.shape: (1, 3, 29)
p: [[[ 0.49847728  0.50165498  0.49770996  0.49649566  0.50189346  0.49644732
    0.49741566  0.4979575   0.50306356  0.49789226  0.50198007  0.50329113
    0.49651733  0.49694782  0.50269896  0.49562088  0.50162542  0.4976519
    0.50289577  0.4970789   0.49805865  0.4989731   0.50193369  0.50472903
    0.50308341  0.50376308  0.49680647  0.49556828  0.49712864]
  [ 0.49821019  0.50229758  0.49791133  0.4947179   0.50194556  0.49783888
    0.4956086   0.4985089   0.50463742  0.49696672  0.50205219  0.50334764
    0.49750876  0.49673349  0.50072753  0.49770546  0.50035363  0.49640232
    0.503968    0.49742669  0.49776521  0.49935529  0.50330091  0.50379097
    0.5033834   0.50246972  0.49803171  0.49633291  0.49783307]
  [ 0.49849835  0.50183225  0.49795225  0.49585757  0.50174952  0.49680921
    0.49627316  0.4979926   0.50433475  0.49716625  0.50238115  0.50417084
    0.49717778  0.49610484  0.50167513  0.49698415  0.50100619  0.4971441
    0.50448686

In [241]:
a = np.array([1,2,3])
print('-------------')
print(a.shape)
print(a)
b = np.expand_dims(a, -1)
print('-------------')
print(b.shape)
print(b)
c =np.expand_dims(a, 0)
print('-------------')
print(c.shape)
print(c)
d =np.expand_dims(a, 1)
print('-------------')
print(d.shape)
print(d)


a[np.newaxis]

-------------
(3,)
[1 2 3]
-------------
(3, 1)
[[1]
 [2]
 [3]]
-------------
(1, 3)
[[1 2 3]]
-------------
(3, 1)
[[1]
 [2]
 [3]]


array([[1, 2, 3]])

In [247]:
a = np.array([[1,2,3], [4,5, 6]], dtype=np.int32, )

In [248]:
a.shape

(2, 3)

In [255]:
print( np.expand_dims(a, -1).shape )
print( np.expand_dims(a, 0).shape )
print( np.expand_dims(a, 1).shape )

(2, 3, 1)
(1, 2, 3)
(2, 1, 3)


In [289]:
a = np.array( [1,2,3,4,5,6,7,8,9,11,22,33,44,55,66,77,88,99] )

In [290]:
a

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 22, 33, 44, 55, 66, 77, 88,
       99])

In [291]:
a.shape = (3,3,2)

In [292]:
a

array([[[ 1,  2],
        [ 3,  4],
        [ 5,  6]],

       [[ 7,  8],
        [ 9, 11],
        [22, 33]],

       [[44, 55],
        [66, 77],
        [88, 99]]])

In [270]:
np.expand_dims(a, -1).shape

(3, 3, 1)

In [293]:
a[1,:,:]

array([[ 7,  8],
       [ 9, 11],
       [22, 33]])

In [294]:
a[:,1,:]

array([[ 3,  4],
       [ 9, 11],
       [66, 77]])

In [296]:
a[:,:,0]

array([[ 1,  3,  5],
       [ 7,  9, 22],
       [44, 66, 88]])

In [311]:
b = np.array( [[[1,2,3],[4,5,6]],
               
               [[7,8,9],[10,11,12]],
               
               [[13,14,15],[16,17,18]],
               
               [[19,20,21],[22,23,24]],
               
               [[25,26,27],[28,29,30]]] )

In [312]:
b.shape

(5, 2, 3)

In [310]:
b.flatten()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [371]:
c = np.array( [[[1,1],[2,2]],
               [[3,3],[4,4]],
               [[5,5],[6,6]]])

In [372]:
c.shape

(3, 2, 2)

In [373]:
print(np.expand_dims(c, -1).shape) # very last axis
print(np.expand_dims(c, 0).shape) # very first axis
print(np.expand_dims(c, 1).shape)
print(np.expand_dims(c, 2).shape)

(3, 2, 2, 1)
(1, 3, 2, 2)
(3, 1, 2, 2)
(3, 2, 1, 2)


In [374]:
d = c[np.newaxis]

In [375]:
d.shape

(1, 3, 2, 2)